In [2]:
#Importar librerías necesarias
import pandas as pd
import numpy as np

# Para exploración de datos:
import matplotlib.pyplot as plt

# Para geodistancias:
from geopy.distance import geodesic

# --------------------------------------------

# Cargar datasets
customers = pd.read_csv('olist_customers_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')

# --------------------------------------------

# Primer vistazo a los datos
print(f"Customers: {customers.shape}")
print(f"Orders: {orders.shape}")
print(f"Geolocation: {geolocation.shape}")

# --------------------------------------------

# Merge de datasets relevantes
# Queremos ubicar cada pedido en su dirección de entrega (ciudad São Paulo)

# Unir orders con customers vía customer_id
orders_customers = pd.merge(orders, customers, on='customer_id', how='inner')

# --------------------------------------------

# Filtrar clientes de São Paulo
orders_sp = orders_customers[orders_customers['customer_city'].str.lower() == 'sao paulo']

print(f"Pedidos en São Paulo: {orders_sp.shape[0]} pedidos")

# --------------------------------------------

# Asignar coordenadas a cada cliente (última dirección conocida)
# Geolocation dataset puede tener varias coordenadas por zip_code, tomaremos el promedio (opcionalmente se puede refinar más)

geo_mean = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

# Truncamos el zip code en customers para hacer el join correctamente (se usa el prefix)
# No creamos 'customer_zip_code_prefix', ya viene en el dataset original
# Merge con coordenadas promedio
orders_sp_geo = pd.merge(
    orders_sp,
    geo_mean,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)


# Merge de coordenadas promedio a pedidos São Paulo
orders_sp_geo = pd.merge(orders_sp, geo_mean, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

# Mostrar un ejemplo de datos resultantes
orders_sp_geo[['order_id', 'customer_unique_id', 'customer_city', 'geolocation_lat', 'geolocation_lng']].head()


Customers: (99441, 5)
Orders: (99441, 8)
Geolocation: (1000163, 5)
Pedidos en São Paulo: 15540 pedidos


,order_id,customer_unique_id,customer_city,geolocation_lat,geolocation_lng
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,sao paulo,-23.576983,-46.587161
1,34513ce0c4fab462a55830c0989c7edb,782987b81c92239d922aa49d6bd4200b,sao paulo,-23.601856,-46.608910
2,5ff96c15d0b717ac6ad1f3d77225a350,e2dfa3127fedbbca9707b36304996dab,sao paulo,-23.713190,-46.687407
3,432aaf21d85167c2c86ec9448c4e42cc,04cf8185c71090d28baa4407b2e6d600,sao paulo,-23.429710,-46.794230
4,203096f03d82e0dffbc41ebc2e2bcfb7,d699688533772c15a061e8ce81cb56df,sao paulo,-23.572939,-46.651115


In [3]:
# Revisar las fechas disponibles en los datos originales
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
print(f"Rango de fechas de pedidos: {orders['order_purchase_timestamp'].min()} → {orders['order_purchase_timestamp'].max()}")

# Merge con customers de nuevo para incluir `order_purchase_timestamp`
orders_customers_full = pd.merge(orders, customers, on='customer_id', how='inner')

# Filtramos São Paulo
orders_customers_sp = orders_customers_full[orders_customers_full['customer_city'].str.lower() == 'sao paulo']

# Seleccionar una fecha específica (ejemplo: '2018-06-01')
fecha_objetivo = '2018-06-01'
pedidos_fecha = orders_customers_sp[orders_customers_sp['order_purchase_timestamp'].dt.date == pd.to_datetime(fecha_objetivo).date()]

print(f"Pedidos en São Paulo en fecha {fecha_objetivo}: {pedidos_fecha.shape[0]}")

# Selección manual del número de pedidos
n_pedidos = 30  # Número que define el usuario
pedidos_seleccionados = pedidos_fecha.sample(n=min(n_pedidos, pedidos_fecha.shape[0]), random_state=42)

print(f"Pedidos seleccionados para optimización: {pedidos_seleccionados.shape[0]}")

# Ahora repetimos merge con coordenadas para estos pedidos
pedidos_seleccionados = pd.merge(
    pedidos_seleccionados,
    geo_mean,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)

# Ver resultado
pedidos_seleccionados[['order_id', 'customer_unique_id', 'customer_city', 'geolocation_lat', 'geolocation_lng']].head()


Rango de fechas de pedidos: 2016-09-04 21:15:19 → 2018-10-17 17:30:18
Pedidos en São Paulo en fecha 2018-06-01: 26
Pedidos seleccionados para optimización: 26


,order_id,customer_unique_id,customer_city,geolocation_lat,geolocation_lng
0,62208055aa533094c967cc9536ecd5a7,36e7ce4e8cfa253aa98130da8a8855bd,sao paulo,-23.595608,-46.643272
1,f6159576a3c6447b217c3b0397b97e06,6077db684bfa5fae744d9a732bce623e,sao paulo,-23.700999,-46.634987
2,c16e485cad605e6180b08e988e5b5bbb,cfd8e25e492965efb735946dba2f2daf,sao paulo,-23.751070,-46.701769
3,db5a3eab52ef5c18c745c172a5050809,3a03d9a7faee99e478d1bd08fa20ee8f,sao paulo,-23.651294,-46.766853
4,b4c4d00d5455e74ead9715bc1f9e8774,18f606b7f084a2dc9cc7578b1c1a0d25,sao paulo,-23.566922,-46.687446


In [4]:
from geopy.distance import geodesic
import numpy as np

# Definir almacén central (coordenadas ficticias en Av. Paulista)
almacen_coord = (-23.561684, -46.656139)

# Construir lista de ubicaciones: primero el almacén, luego los pedidos
locations = [almacen_coord] + list(zip(pedidos_seleccionados['geolocation_lat'], pedidos_seleccionados['geolocation_lng']))

n_locations = len(locations)
print(f"Total de ubicaciones (incluyendo almacén): {n_locations}")

# Calcular matriz de distancias en km
distance_matrix_km = np.zeros((n_locations, n_locations))

for i in range(n_locations):
    for j in range(n_locations):
        if i != j:
            distance_matrix_km[i][j] = geodesic(locations[i], locations[j]).km

# Convertir a matriz de tiempos (minutos)
velocidad_media_kmh = 30  # supuesta velocidad media urbana
distance_matrix_min = (distance_matrix_km / velocidad_media_kmh) * 60  # tiempo en minutos

# Mostrar una muestra de la matriz
print("Matriz de tiempo estimado (minutos) [primeras 5 filas]:")
print(distance_matrix_min[:5, :5])


Total de ubicaciones (incluyendo almacén): 27
Matriz de tiempo estimado (minutos) [primeras 5 filas]:
[[ 0.          7.96027692 31.15951752 42.97088616 30.07805282]
 [ 7.96027692  0.         23.40587254 36.44518759 28.07653013]
 [31.15951752 23.40587254  0.         17.56386505 29.06782757]
 [42.97088616 36.44518759 17.56386505  0.         25.78160202]
 [30.07805282 28.07653013 29.06782757 25.78160202  0.        ]]


In [6]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

# Definir parámetros
tiempo_servicio = 8  # minutos por cliente
capacidad_maxima = 10  # pedidos máximos por vehículo/ruta
tiempo_max_ruta = 480  # tiempo máximo de ruta (minutos)

num_vehiculos = 10  # número máximo de vehículos permitidos
depot = 0  # nodo de inicio (almacén central)

# Crear Index Manager y Routing Model
manager = pywrapcp.RoutingIndexManager(len(distance_matrix_min), num_vehiculos, depot)
routing = pywrapcp.RoutingModel(manager)

# Callback de tiempo con servicio incluido
def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    tiempo_viaje = int(distance_matrix_min[from_node][to_node])
    if from_node != depot:
        tiempo_viaje += tiempo_servicio  # sumar tiempo de servicio antes de salir del nodo
    return tiempo_viaje

transit_callback_index = routing.RegisterTransitCallback(time_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Añadir dimensión de tiempo
routing.AddDimension(
    transit_callback_index,
    0,  # sin holgura
    tiempo_max_ruta,  # tiempo máximo permitido por vehículo
    True,  # tiempo acumulado empieza en 0
    'Time'
)

time_dimension = routing.GetDimensionOrDie('Time')

# Añadir restricción de capacidad
demanda = [0] + [1] * (len(distance_matrix_min) - 1)  # 1 pedido por cliente, 0 en almacén

def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return demanda[from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # sin holgura
    [capacidad_maxima] * num_vehiculos,  # misma capacidad para todos los vehículos
    True,  # inicia en 0
    'Capacity'
)

# Configuración del solver
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.time_limit.seconds = 30
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

# Resolver
solution = routing.SolveWithParameters(search_parameters)

# Mostrar resultados
if solution:
    print("✅ Solución encontrada:")
    for vehicle_id in range(num_vehiculos):
        index = routing.Start(vehicle_id)
        route = []
        route_time = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                route_time += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        if len(route) > 1:
            print(f"🚌 Vehículo {vehicle_id}: ruta {route} con tiempo estimado {route_time} minutos")
else:
    print("⚠️ No se encontró solución en el tiempo límite.")


✅ Solución encontrada:
🚌 Vehículo 7: ruta [0, 1, 20, 21, 10, 25, 6, 15, 22, 11, 12] con tiempo estimado 182 minutos
🚌 Vehículo 8: ruta [0, 16, 17, 24, 18, 7, 13, 23] con tiempo estimado 99 minutos
🚌 Vehículo 9: ruta [0, 19, 14, 26, 2, 8, 3, 4, 9, 5] con tiempo estimado 166 minutos


In [9]:
import folium

# 📍 Centro del mapa en almacén
m = folium.Map(location=almacen_coord, zoom_start=12)

# Marcador del almacén
folium.Marker(
    location=almacen_coord,
    popup='Almacén Central',
    icon=folium.Icon(color='red', icon='home')
).add_to(m)

# 🎨 Colores para rutas (más largo para garantizar variedad)
colors = [
    'blue', 'green', 'purple', 'orange', 'darkred',
    'cadetblue', 'darkgreen', 'black', 'darkpurple', 'lightblue'
]

# Recorremos las rutas para todos los vehículos usados
for vehicle_id in range(num_vehiculos):
    index = routing.Start(vehicle_id)
    route_coords = []
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        coord = locations[node_index]
        route_coords.append(coord)

        # Marcar clientes (omitimos almacén en iconos truck)
        if node_index != 0:
            folium.Marker(
                location=coord,
                popup=f"Pedido {node_index}",
                icon=folium.Icon(color=colors[vehicle_id % len(colors)], icon='truck')
            ).add_to(m)

        index = solution.Value(routing.NextVar(index))

    # Añadir la línea de ruta si visitó al menos un cliente
    if len(route_coords) > 1:
        folium.PolyLine(
            locations=route_coords,
            color=colors[vehicle_id % len(colors)],
            weight=4,
            opacity=1,
            popup=f'Ruta Vehículo {vehicle_id}'
        ).add_to(m)

# Mostrar mapa
m
